# Introduction to the Reverse Osmosis Unit Model
This tutorial shows how to build, initialize, simulate, and optimize a reverse osmosis (RO) unit model using WaterTAP. In this example, we will minimize the specific energy consumption of the RO unit.

In [1]:
from pyomo.environ import ConcreteModel, Objective, Expression, value
from idaes.core.util.model_statistics import degrees_of_freedom

# Python magics from dsl.py
%load_ext dsl
# Utility functions to retrieve result objects
from dsl import get_model, get_results

### Specify values and scale for system variables

In [3]:
%%iic
start
# Step 1. create unit
unit RO-0D
# Step 2. fix values
fix inlet flow mass NaCl 0.035
fix inlet flow mass H2O 0.965
fix inlet pressure 50e5
fix inlet temperature 298.15
fix membrane area 50
fix membrane permeability water 4.2e-12
fix membrane permeability salt 3.5e-8
fix permeate pressure 101325
# Step 3. add scaling
scale flow mass NaCl 1e2
scale flow mass H2O 1
# Step 4. Initialize the model
init
# Step 5. Solve
solve

2022-02-02 09:50:18 [WARNING] idaes.core.util.scaling: Missing scaling factor for fs.unit.area
2022-02-02 09:50:19 [INFO] idaes.init.fs.unit: Initialization Step 1 Complete.
2022-02-02 09:50:19 [INFO] idaes.init.fs.unit: Initialization Step 2 Complete.
2022-02-02 09:50:19 [INFO] idaes.init.fs.unit: Initialization Step 3 successful.
2022-02-02 09:50:19 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found


In [18]:
get_model().fs.unit.display()

Block fs.unit

  Variables:
    A_comp : Solvent permeability coeff.
        Size=1, Index=fs.unit.A_comp_index, Units=m**2*s/kg
        Key          : Lower : Value   : Upper : Fixed : Stale : Domain
        (0.0, 'H2O') : 1e-18 : 4.2e-12 : 1e-06 :  True :  True : NonNegativeReals
    B_comp : Solute permeability coeff.
        Size=1, Index=fs.unit.B_comp_index, Units=m/s
        Key           : Lower : Value   : Upper : Fixed : Stale : Domain
        (0.0, 'NaCl') : 1e-11 : 3.5e-08 : 1e-05 :  True :  True : NonNegativeReals
    area : Membrane area
        Size=1, Index=None, Units=m**2
        Key  : Lower : Value             : Upper : Fixed : Stale : Domain
        None :   1.0 : 143.9451916201364 : 500.0 : False : False : NonNegativeReals
    recovery_mass_phase_comp : Mass-based component recovery
        Size=2, Index=fs.unit.recovery_mass_phase_comp_index
        Key                  : Lower : Value                : Upper    : Fixed : Stale : Domain
         (0.0, 'Liq', 'H2O'

### Unfix variables

In [5]:
%%iic
# Unfix and set lower and upper bounds for membrane area (m^2)
unfix membrane area bounds 1 500
# Unfix and set lower and upper bounds for feed pressure (Pa)
unfix inlet pressure bounds 10e5 80e5

### Add objective and constraint for minimizing specific energy consumption, with water recovery at 50%

In [9]:
m = get_model()
# Assume 100% efficiency of pumps and ERD and no pressure losses
#--> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0]/(3.6e6))

In [10]:
# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

In [11]:
# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0,'Liq'].fix(0.50)

In [12]:
# The solver will find the membrane area and 
# inlet pressure that achieve 50% recovery while minimizing
# specific energy consumption. Since we fixed the 
# volumetric water recovery, a degree of freedom 
# was removed from the model.
print(degrees_of_freedom(m))

1


### Run optimization

In [13]:
# optimization_results = solver.solve(m)
%ii solve

In [14]:
# membrane area of the optimized RO unit
value(m.fs.unit.area)

143.9451916201364

In [15]:
# inlet pressure of the optimized RO unit
value(m.fs.unit.inlet.pressure[0])

4526838.374296251

In [16]:
# the minimum specific energy consumption
value(m.fs.specific_energy_consumption)

1.2574551039711808

In [17]:
# display the overall report on the RO unit
m.fs.unit.report()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                                            : Value      : Fixed : Bounds
                                     Membrane Area :     143.95 : False : (1.0, 500.0)
                   NaCl Concentration @Inlet,Bulk  :     35.751 : False : (0.001, 2000.0)
     NaCl Concentration @Inlet,Membrane-Interface  :     35.751 : False : (0.001, 2000.0)
                  NaCl Concentration @Outlet,Bulk  :     70.960 : False : (0.001, 2000.0)
    NaCl Concentration @Outlet,Membrane-Interface  :     70.960 : False : (0.001, 2000.0)
                      NaCl Permeate Concentration  :    0.45419 : False : (0.001, 2000.0)
        Osmotic Pressure @Inlet,Membrane-Interface : 2.8528e+06 : False : (500.0, 50000000.0)
                     Osmotic Pressure @Outlet,Bulk : 5.8432e+06 : False : (500